# Julia for Economists
## Parallelization for Fun and Profit
### Cameron Pfiffer (cpfiffer@stanford.edu)


## Brief introduction

- I am Cameron Pfiffer, howdy
- Finance PhD student at University of Oregon
- Visiting at the Stanford GSB
- Work mostly on asset pricing, Bayesian econometrics, and computer go-fast stuff


## What is this?

- Second in a five-part series on Julia (don't worry if you missed the first)
    3. Optimization and Automatic Differentiation (March)
    4. Performant Programming and Best Practices (April)
    5. Bayesian Inference and Probabilistic Programming (May)
- The focus is programming better for research computing


## What are we going to do today?

- Learn all about the CPU!
- Multithreading
- Multiprocessing
- GPU parallelism


## How are we going to things do today?

- 50 minute lectures.
- 10 minute breaks at the top of the hour, 15 minutes at 11am.
- Interrupt me to ask questions, or type them in the chat
- Intermittent breaks for you to practice some code

## Preliminaries

You will need

1. A Julia installation
2. A text editor (VS Code recommended)
3. Your big beautiful brain

If you do not have one of these **please take a moment to go get them**. 

You have a big beautiful brain, don't lie to me about that.


## Warning!

I learned a lot of this informally -- parallelization is typically a very dense and technical concept.

I'm going to try my best to give you rigor when I can, but much of this will be a folksy introduction to parallel computing!


# Questions, comments, concerns?


# The modern central processing unit and how amazing it is


## CPUs

You might be aware that there's lots of little bits in your computer that do various kinds of magic.

The CPU is easily one of the most important ones for research computing! And it is full of magic!


## CPUs

CPUs are fancy boxes that accept

1. _Instructions_ on what to do (machine code)
2. _Data_ to use for the instructions (numbers, pixels, etc.)

and perform the many requested tasks we ask them to do.


## What's inside your CPU (the important conceptual ones)?

- The __control unit (CU)__, which turns binary into timing and control signals.
- The __arithmatic logic unit (ALU)__, which does "the math".
- The __cache__, which is a place to store data and instructions for quick access. The cache stores quick-to-access versions of what is in main memory (RAM).
- The __registers__, tiny little caches that the ALU can get to directly.


## What is inside your CPU?

![](https://www.redhat.com/sysadmin/sites/default/files/styles/embed_large/public/2020-07/Figure2.png?itok=mQGilPbJ)

[Image source](https://www.redhat.com/sysadmin/cpu-components-functionality).

## What is a "core"

CPUs are composed of one or more smaller **cores**. For example, a dual-core processor has two ALUs, one or more CUs, and may have caches unique to them (though commonly L2 or L3 are shared).

Multiple cores can do totally differently things without impacting each other too much!

## The cache

The cache is short-term storage for instructions and data. It follows an organizational hierarchy:

- L1 is a small cache that is very close to the processor. It is _really_ fast.
- L2 is a larger cache that's a bit slower.
- L3 is a specialzed cache designed to make L1 and L2 work better.

When the processor needs data, it starts with L1, then L2, then L3, and if it is not in the CPU cache, it will check main memory (RAM).

## Checking main memory is slow

Here's [a typical latency](https://www.technipages.com/what-is-the-cpu-cache) for L1, L2, L3, and main memory:

- L1: 0.8 nanoseconds
- L2: 2.4 nanoseconds
- L3: 11.1 nanoseconds
- Main memory: 14 nanoseconds

This is a long time! If stuff is already in your processor, it will be fast. If it has to go outside the cache, you may have to wait.

This is why you may have heard of _vectorization_ being fast, especially if you come from an R world where vectoriazation is a good way to get free speed.

Vectorization means you can keep the instruction (`sin`, for example) in L1 (or the register) and pump data through, meaning you don't have to do cache checks for new instructions.

## CPU internals (cache misses)

For our purposes, the important parts are the three cache levels (L1, L2, L3). 

These store instructions and data. When the ALU needs data to do some math (`x + y`), it has to ask the cache in order if it has the data available -- if not, you may need to go get it from RAM, which is very costly!

Not having the data in a cache and having to retrieve it is called a _cache miss_, and can be an important computational cost. For our purposes, these costs are quite small and likely to be difficult to track down. 

Don't worry about them too much.

## Serial processing

CPUs at their most basic level do these tasks in *serial*, meaning they do them one at a time.

If we have a long list of things we want done, it will do them _in order_ -- the sixth task is only started after the fifth is done.

## Serial processing

If, for example, I ran the following

```julia
x = 0
for i in 1:2
    x = x + i
end
```

we know we'll have `x=3` at the end, but we executed the pseudo-machine code

```julia
x = 0
x = x + 1
x = x + 2
```

This is the order of the code, and it will _always_ be the order of the code.


## Serial processing

There's a couple wonderful things about serial processing:

1. It is deterministic (always know what you're going to get)
2. CPUs are super fast at serial processing
3. Lots of complex hidden benefits (easier to cache instructions/data, etc.)

Most things are serial by default. You have to do a bit of work to make things parallel/concurrent!

## Serial processing

The downside is that serial processing requires __unrelated tasks__ to be completed _in order_!

This is fine if I'm doing something like

```julia
function f(a, b)
    x = 0
    y = 0
    for i in 1:10
        x += a * i
        y += b * i
    end
    
    return x, y
end
```

since calculating `x = x + a*i` is hilariously cheap. 

Note that we do not update `y` without first updating `x`, even though the _values_ of `x` and `y` do not depend on each other (only `i`).

## Serial processing

Serial computation can be extremely costly if you are doing lots of costly things in order. Take the following:

In [35]:
using LinearAlgebra, BenchmarkTools

function expensive()
    X = randn(10_000, 100)
    return X'X
end

function go()
    A = expensive()
    B = expensive() # Have to wait for the first call to finish!
    return A + B
end

# @benchmark runs expensive a bunch of times to estimate
# the computational run time.
@benchmark go()

BenchmarkTools.Trial: 330 samples with 1 evaluation.
 Range (min … max):  13.140 ms … 28.532 ms  ┊ GC (min … max): 0.00% … 3.31%
 Time  (median):     13.844 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.153 ms ±  2.780 ms  ┊ GC (mean ± σ):  2.44% ± 3.72%

  █▃▁▁▂  ▁▄▁                       ▄                           
  █████▇▆███▇▆▇▅▅▆▄▁▁▅▄▄▄▅▅▁▁▁▁▄▁▁▁█▇▆▇▅▄▅▁▄▁▄▁▅▄▄▁▅▄▁▁▁▄▁▁▁▄ ▆
  13.1 ms      Histogram: log(frequency) by time        24 ms <

 Memory estimate: 15.49 MiB, allocs estimate: 10.

In the above function, we might want to compute `A` and `B` _separately_ at the same time (since they do not depend on one another) and then handle the sum component at the end.

In other words, how to we get away from this serial computation paradigm? Is there a way so that I don't have to wait for an expensive function to complete before I move on to the next?

## Multiprocess & multithread parallelism

There are two common ways of achieving computational parallelism. 

- Multithreading, where you use one or more threads as part of a _process_. All threads share memory with one another.
- Multiprocess, where you create an entire new process that has its own memory footprint. The new process(es) may have multiple threads.

Multithreading is cheaper and lighter, but multiprocess parallelism is more general and extensible.

__I generally prefer to multithread where possible__ -- multiprocess programming is often a lot harder, though it can be a spectacular fit for a class of problems.

## Multithreading

Multithreading is (one) way to alleviate the friction of having to wait for sequential expressions to complete.

You can think of a thread as a friend you've asked to help you move -- you can give them top-level instructions such as "move all the boxes from the living room into the truck", which frees you up to go pack boxes in the bedroom.

## Multiprocessing

To stick with the moving example, I might think of multiprocessing as hiring several moving companies! Each company has their own workers and goals, all (hypothetically) working independently from one another.

I will get to this later -- but for now, let's focus on multithreading.



## Starting Julia with threads

In order to use any threading capabilities in Julia, you have to create a Julia process with threads at the start. How to set this depends on how you are using Julia:

- **Command line**. Set the threads with `julia --threads=n_threads`, or default to the number of cores you have with `julia --threads=auto`.
- **VS Code REPL**. Go to the Julia extension settings, search for "threads" and enter a number -- 4 is probably safe, or use the number of cores in your machine if you know how many you have.

Don't launch a ton of threads! You'll start getting real slow if you generate too many, and you may even lock your machine up. `--threads=auto` is safest for most people.

Once inside Julia, run `Threads.nthreads()` and check to see that the result is greater than one.

If in Jupyter, you will need to make a new kernel:

```julia
using IJulia
IJulia.installkernel(
    "Julia multithreading", 
    env=Dict("JULIA_NUM_THREADS" => 4)
)

```

Once we have multiple threads, we can do all kinds of cool stuff! 

Let's start with just seeing how to use the more common threading construct, a multithreaded `for` loop.

To do this, you can simply put `Threads.@threads` in front of `for`, and Julia will dispatch threads to handle different components of the loop.

In [27]:
# Threads.threadid() tells us the ID number of the thread currently doing a task.
function thread_test()
    Threads.@threads for i in 1:6
        id = Threads.threadid()
        println("Thread ID: ", id, " | i: ", i)
    end
end

thread_test()

Thread ID: 3 | i: 5
Thread ID: 1 | i: 1
Thread ID: 2 | i: 3
Thread ID: 1 | i: 2
Thread ID: 2 | i: 4
Thread ID: 4 | i: 6


## Extremely important thing to see!

The order is not deterministic! 

Threads show up differently when you re-run the code, and they are assigned different parts of the looping variable.

If you start working with threads, you need to be aware that you are assigning a large _bundle_ of tasks, and that you cannot guarantee the order in which the tasks are completed.